In [1]:
import sys
import scanpy as sc

sys.path.append("../..")
from scripts.runner_models import cv_dca_5fold

In [ ]:
import sys, types
import tensorflow.keras.optimizers.legacy as legacy_opt

# Build a tiny shim module so `from keras.optimizers import RMSprop` etc. pick legacy classes
shim = types.ModuleType("keras.optimizers")
for name in ("SGD", "RMSprop", "Adam", "Adagrad", "Adadelta", "Adamax", "Nadam", "Ftrl", "Optimizer"):
    if hasattr(legacy_opt, name):
        setattr(shim, name, getattr(legacy_opt, name))

# Ensure our shim is used for future imports
sys.modules.pop("keras.optimizers", None)   # in case it's already imported
sys.modules["keras.optimizers"] = shim

# (Optional) make sure GPU mem growth is on
try:
    import tensorflow as tf
    for g in tf.config.list_physical_devices("GPU"):
        tf.config.experimental.set_memory_growth(g, True)
except Exception:
    pass


2025-09-17 19:17:29.866817: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-17 19:17:29.916590: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-09-17 19:17:29.916665: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-09-17 19:17:29.918658: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-17 19:17:29.929325: I tensorflow/core/platform/cpu_feature_guar

In [3]:
# Load data
IN_PATH_neu = "../../data/raw_count/GSE169569_raw_counts.h5ad"
IN_PATH_cov = "../../data/raw_count/GSE228841_raw_counts.h5ad"

adata_neu = sc.read_h5ad(IN_PATH_neu)
adata_cov = sc.read_h5ad(IN_PATH_cov)

In [4]:
# --- PATCH PyYAML load() to default to a safe/full loader if none was given ---
import yaml

if hasattr(yaml, "load"):
    _orig_load = yaml.load
    def _patched_load(stream, Loader=None, *args, **kwargs):
        if Loader is None:
            try:
                from yaml import FullLoader
                Loader = FullLoader
            except Exception:
                from yaml import SafeLoader
                Loader = SafeLoader
        return _orig_load(stream, Loader=Loader, *args, **kwargs)
    yaml.load = _patched_load

# Sanity check: should NOT raise TypeError now
assert yaml.load("a: 1") == {"a": 1}


In [5]:
import numpy as np

# Extract data matrices and metadata
X_counts = adata_neu.layers["counts"] # raw counts
gene_names = np.array(adata_neu.var_names, dtype=str)
batches = np.array(adata_neu.obs["BioProject"], dtype=str)
norm_layer = adata_neu.layers["log2_1p_CPM_original"]

In [6]:
import optuna

def objective(trial: optuna.Trial):
    hidden_size = [
        trial.suggest_categorical("h1", [32, 64, 128]),
        trial.suggest_categorical("h2", [16, 32, 64]),
        trial.suggest_categorical("h3", [32, 64, 128]),
    ]
    epochs = trial.suggest_categorical("epochs", [50, 100, 150])
    batch_size = trial.suggest_categorical("batch_size", [64, 128])
    n_hvg = trial.suggest_categorical("n_hvg", [800, 1000, 1500, 2000])

    summary, _ = cv_dca_5fold(
        X_counts, k=3,
        dca_params=dict(hidden_size=hidden_size, epochs=epochs, batch_size=batch_size),
        n_hvg=int(n_hvg), R=1,
        mask_frac=0.10, thinning_p=0.10, random_state=123,
        hvg_mode="seurat_v3",
        gene_names=gene_names, batches=batches,
        norm_layer=norm_layer, batch_key="BioProject",
        seurat_layer_name="log2_1p_CPM_original",
        save_dir=None
    )
    # choose your target metric
    return float(summary["NB_ll_zero"].mean())

study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=123))
study.optimize(objective, n_trials=20, gc_after_trial=True)

print("Best params:", study.best_params)

# Final retrain with “fuller” settings
best = study.best_params
final_summary, final_details = cv_dca_5fold(
    X_counts, k=5,
    dca_params=dict(hidden_size=[best["h1"], best["h2"], best["h3"]],
                    epochs=300, batch_size=best["batch_size"]),
    n_hvg=int(best["n_hvg"]), R=3,
    mask_frac=0.10, thinning_p=0.10, random_state=123,
    hvg_mode="seurat_v3",
    gene_names=gene_names, batches=batches,
    norm_layer=norm_layer, batch_key="BioProject",
    seurat_layer_name="log2_1p_CPM_original",
    save_dir="results/dca_full"
)

/home/ma/ma_ma/ma_minjlee/.conda/envs/dca_gpu/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-09-17 13:16:33,470] A new study created in memory with name: no-name-84290c80-a98f-47e0-b7d0-fb105ed76f24


2025-09-17 13:16:33 | [cv_dca_5fold] start k=3 n_hvg=1000 mode=seurat_v3 R=1 mask_frac=0.1 thinning_p=0.1
2025-09-17 13:16:33 | X_counts: dense  (447, 51777) dtype=int64
2025-09-17 13:17:03 | X_counts: zeros=66.23%, approx_int=True
2025-09-17 13:17:03 | [time] [DCA] fold 1/3 start
2025-09-17 13:17:03 | X_tr: dense  (298, 51777) dtype=int64
2025-09-17 13:17:03 | X_va: dense  (149, 51777) dtype=int64


/home/ma/ma_ma/ma_minjlee/.conda/envs/dca_gpu/lib/python3.10/site-packages/scanpy/preprocessing/_highly_variable_genes.py:64: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  warnings.warn(


2025-09-17 13:17:45 | [DCA] fold 1 | HVG=1000
2025-09-17 13:17:45 | [DCA] fit hidden=[32, 32, 32] epochs=150 bs=64
2025-09-17 13:17:45 | X_train: dense  (298, 1000) dtype=int64
2025-09-17 13:17:45 | [time] [DCA] fold 1 predict mu_va start


/home/ma/ma_ma/ma_minjlee/.conda/envs/dca_gpu/lib/python3.10/site-packages/hyperopt/atpe.py:19: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


...

2025-09-17 13:54:02 | [DCA] fold 5 | HVG=2000
2025-09-17 13:54:02 | [DCA] fit hidden=[32, 64, 128] epochs=300 bs=64
2025-09-17 13:54:02 | X_train: dense  (358, 2000) dtype=int64
2025-09-17 13:54:02 | [time] [DCA] fold 5 predict mu_va start
2025-09-17 13:54:02 | [DCA] predict_mean
2025-09-17 13:54:02 | [time] [DCA] train(dca.api.dca) start
dca: Successfully preprocessed 2000 genes and 447 cells.


2025-09-17 13:54:02.363078: W tensorflow/c/c_api.cc:305] Operation '{name:'batch_normalization_2/beta/Assign' id:174 op device:{requested: '', assigned: ''} def:{{{node batch_normalization_2/beta/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](batch_normalization_2/beta, batch_normalization_2/beta/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2025-09-17 13:54:02.818924: W tensorflow/c/c_api.cc:305] Operation '{name:'batch_normalization_2/cond_3/Identity' id:205 op device:{requested: '', assigned: ''} def:{{{node batch_normalization_2/cond_3/Identity}} = Identity[T=DT_FLOAT, _has_manual_control_dependencies=true](batch_normalization_2/cond_3)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and w

dca: Calculating reconstructions...
2025-09-17 13:54:29 | [time] [DCA] train(dca.api.dca) end in 27.36s
2025-09-17 13:54:29 | [time] [DCA] fold 5 predict mu_va end in 27.36s


/home/ma/ma_ma/ma_minjlee/.conda/envs/dca_gpu/lib/python3.10/site-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
2025-09-17 13:54:29.374102: W tensorflow/c/c_api.cc:305] Operation '{name:'slice/slice/Identity' id:303 op device:{requested: '', assigned: ''} def:{{{node slice/slice/Identity}} = Identity[T=DT_FLOAT, _has_manual_control_dependencies=true](lambda_1/mul)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


2025-09-17 13:54:29 | [time] [DCA] fold 5/5 end in 38.30s
2025-09-17 13:54:29 | [cv_dca_5fold] wrote CSVs to results/dca_full
2025-09-17 13:54:29 | [cv_dca_5fold] done


In [8]:
final_summary

,model,n_hvg,MAE_zero,MSE_zero,MedianL1_zero,NB_ll_zero,NB_dev_zero,MAE_thin,MSE_thin,MedianL1_thin,NB_ll_thin,NB_dev_thin,Silhouette,params
0,DCA,2000,140.469869,6.946264e+06,1.99856,-4.398858,0.176755,7.171667,42724.766382,0.041514,-0.906699,0.163444,NaN,"{'batch_size': 64, 'epochs': 300, 'hidden_size..."


In [9]:
final_details

,fold,model,params,n_hvg,MAE_zero,MSE_zero,MedianL1_zero,NB_ll_zero,NB_dev_zero,MAE_thin,MSE_thin,MedianL1_thin,NB_ll_thin,NB_dev_thin,Silhouette
0,1,DCA,"{'hidden_size': (32, 64, 128), 'epochs': 300, ...",2000,151.627812,9.046060e+06,2.056759,-4.407768,0.191263,8.277194,44330.547184,0.042287,-0.926098,0.168387,None
1,2,DCA,"{'hidden_size': (32, 64, 128), 'epochs': 300, ...",2000,121.145013,3.525893e+06,2.009062,-4.420139,0.164807,6.516910,30828.864925,0.041377,-0.898324,0.157701,None
2,3,DCA,"{'hidden_size': (32, 64, 128), 'epochs': 300, ...",2000,148.512201,7.308547e+06,1.878331,-4.345718,0.167459,7.243993,35116.322526,0.038614,-0.863282,0.157218,None
3,4,DCA,"{'hidden_size': (32, 64, 128), 'epochs': 300, ...",2000,157.543907,9.801430e+06,2.122466,-4.468428,0.172459,7.663326,79929.623648,0.047118,-0.974785,0.171977,None
4,5,DCA,"{'hidden_size': (32, 64, 128), 'epochs': 300, ...",2000,123.520415,5.049390e+06,1.926183,-4.352239,0.187789,6.156912,23418.473626,0.038174,-0.871007,0.161936,None
